In [70]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import preprocessing
import graphviz
import numpy as np
from scipy.sparse import csr_matrix
from scipy.spatial.distance import euclidean, pdist, squareform
import random

In [57]:
data_business = pd.read_json("yelp_dataset/yelp_academic_dataset_business.json", lines=True)
data_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{u'BusinessParking': u'{'garage': False, 'stre...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{u'Monday': u'8:30-17:0', u'Tuesday': u'11:0-2...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{u'RestaurantsTableService': u'False', u'GoodF...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{u'Sunday': u'17:0-23:0', u'Friday': u'17:0-23...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{u'RestaurantsTableService': u'True', u'GoodFo...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{u'Monday': u'10:0-22:0', u'Tuesday': u'10:0-2...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{u'BusinessAcceptsCreditCards': u'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{u'Friday': u'8:0-17:0', u'Tuesday': u'8:0-17:...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [58]:
data_user = pd.read_json("yelp_dataset/yelp_academic_dataset_user.json", lines=True)
data_user.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,2.00,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Susan,1,0,lzlZwIpuSWXEnNS91wxjHw,2015-09-28
1,5.00,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Daipayan,2,0,XvLBr-9smbI0m_a7dXtB7w,2015-09-05
2,4.00,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Andy,1,0,QPT4Ud4H5sJVr68yXhoWFw,2016-07-21
3,4.05,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Jonathan,19,0,i5YitlHZpf0B3R0s_8NVuw,2014-08-04
4,3.00,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Shashank,3,0,s4FoIXE_LSGviTHBe8dmcg,2017-06-18


In [59]:
data_review = pd.read_json("yelp_dataset/yelp_academic_dataset_review.json", lines=True)
data_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g


In [93]:
joined_data = pd.merge(data_business, data_review, on="business_id", how="inner", sort=False, suffixes=('_x', '_y'))
#joined_data = pd.merge(joined_data, data_user, on="user_id", how="inner", sort=False, suffixes=('_x', '_y'))
joined_data = joined_data[['business_id','city','categories','user_id','stars_x']]
joined_data.head()

,business_id,city,categories,user_id,stars_x
0,45bWSZtniwPRiqlivpS8Og,Phoenix,"Coffee & Tea, Food",kjfBvZlRzs1_7vdp7SZbeg,4.0
1,45bWSZtniwPRiqlivpS8Og,Phoenix,"Coffee & Tea, Food",h8RmB4pdbXwVVfOQjg21zA,4.0
2,KapTdGyGs7RK0c68Z6hhhg,Markham,"Restaurants, Japanese",rPbIVGuIjmxHDw8ChhTsrw,1.5
3,tZnSodhPwNr4bzrwJ1CSbw,Toronto,"Cajun/Creole, Southern, Restaurants",ibftnXIQUhOvYhWCIyfRng,4.0
4,tZnSodhPwNr4bzrwJ1CSbw,Toronto,"Cajun/Creole, Southern, Restaurants",UkghXQuglpbzGpUhTKCBNg,4.0


In [115]:
training_data, test_data = train_test_split(joined_data, test_size = 0.2)

utility_matrix_training = pd.pivot_table(training_data, index='business_id', columns=['user_id'], values='stars_x', fill_value=0)
utility_matrix_training.columns = range(len(utility_matrix_training.columns.values))
utility_matrix_training = utility_matrix_training.T
utility_matrix_training.columns = range(len(utility_matrix_training.columns.values))

utility_matrix_test = pd.pivot_table(test_data, index='business_id', columns=['user_id'], values='stars_x', fill_value=0)
utility_matrix_test.columns = range(len(utility_matrix_test.columns.values))
utility_matrix_test = utility_matrix_test.T
utility_matrix_test.columns = range(len(utility_matrix_test.columns.values))
pd.DataFrame(utility_matrix_training).head()

,0,1,2,3,4,5,6,7,8,9,...,4269,4270,4271,4272,4273,4274,4275,4276,4277,4278
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
pd.DataFrame(utility_matrix_training).shape

(6153, 4279)

In [117]:
def mse_utility(u1, u2):
    return mse(u1[u1.nonzero()].flatten(), u2[u2.nonzero()].flatten())

def top_k(arr, self_idx, k):
    i = np.argsort(-arr)[:k+1]
    return {index:arr[index] for index in i[1:]}

def create_similarity_matrix_users(u):
    DF_euclid = squareform(pdist(u, metric='euclidean'))
    return DF_euclid


    
similarity_user = create_similarity_matrix_users(utility_matrix_training)
print(similarity_user)

[[0.         4.30116263 5.70087713 ... 4.94974747 5.31507291 4.03112887]
 [4.30116263 0.         5.14781507 ... 4.30116263 4.71699057 3.20156212]
 [5.70087713 5.14781507 0.         ... 5.70087713 6.02079729 4.9244289 ]
 ...
 [4.94974747 4.30116263 5.70087713 ... 0.         5.31507291 4.03112887]
 [5.31507291 4.71699057 6.02079729 ... 5.31507291 0.         4.47213595]
 [4.03112887 3.20156212 4.9244289  ... 4.03112887 4.47213595 0.        ]]


In [119]:
print (similarity_user.shape)

def rec_via_users(m_utility, m_sim_users, user_idx, item_idx, k):
    userslist = top_k(m_sim_users[user_idx], user_idx, k)
    numerator, denominator= [], []
    print(userslist)
    print(m_utility[485][301])
    for k, v in userslist.items():
        if m_utility[k][item_idx] != 0.0:
            numerator.append(m_utility[k,item_idx]*v)
            denominator.append(v)
        else:
            denominator.append(0)
    if(sum(denominator) != 0):
        return sum(numerator) / sum(denominator)
    else:
        return 0
    
def recs_via_users(m_utility, m_sim_users, k, test_n):
    #randomly pick up n test users from the whole users
    test = random.sample(range(m_sim_users.shape[0]), test_n)
#     print(test)
    true = []
    pred = []
    for user_idx in test:
        for item_idx in range(m_utility.shape[1]):
#             print(m_utility[user_idx][item_idx])
            if m_utility[user_idx][item_idx] != 0:
                true.append(m_utility[user_idx][item_idx])
                p=round(rec_via_users(m_utility, m_sim_users, user_idx, item_idx, k))
                if p != 0:    
                    pred.append(p)
                else:
                    pred.append(1.0e-9)
    return mse_utility(np.array([true], dtype=np.float64), np.array([pred], dtype=np.float64))

num_k_userbased = []
mses = []
for i in range(200):
    num_k_userbased.append(i+1)
    mses.append(recs_via_users(utility_matrix_training, similarity_user, i+1, 300))
    print("{}/200".format(i+1), mses[-1])


(6153, 6153)


KeyError: (3394, 0)